In [2]:
import numpy as np

In [3]:
opcodeInput=np.loadtxt("input_day13.txt",delimiter=",",dtype=np.int64)

In [4]:
class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

In [5]:
def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [113]:
class ArcadeGame:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.tiles={}
        self.input=0
        self.score=0
        
    def clearOutput(self):
        self.outputList=[]
    
    def processOutput(self):
        numOutput=len(self.outputList)
        assert(numOutput%3==0)
        numNewCommands=numOutput//3
        for command in np.array(self.outputList).reshape((numNewCommands,3)):
            if command[0] == -1 and command[1] == 0:
                #score
                #print("Score difference: " +str(command[2]-self.score))
                self.score=command[2]
            else:
                #if (command[0],command[1]) in self.tiles:
                #    if self.tiles[(command[0],command[1])] == 2 and command[2] == 0:
                #        print("Erased block from " + str(command[0]) + ", " + str(command[1]))
                self.tiles[(command[0],command[1])] = command[2]
        self.clearOutput()

    def resetTiles(self):
        self.tiles={}
        
    def visualizeTiles(self):
        xMax,yMax=0,0
        for tile in self.tiles:
            xMax=max(xMax,tile[0])
            yMax=max(yMax,tile[1])
        #print(xMax,yMax)
        screen=np.full((yMax+1,xMax+1)," ")
        for entry in self.tiles:
            tileType=self.tiles[entry]
            tileStr=' '
            if tileType == 1:
                tileStr= '#'
            elif tileType == 2:
                tileStr = 'B'
            elif tileType == 3:
                tileStr = '-'
            elif tileType == 4:
                tileStr = 'o'
            screen[entry[1],entry[0]] = tileStr
        outputStr=''
        for j,row in enumerate(screen):
            for elem in row:
                outputStr += elem
            outputStr += '\n'
        print(outputStr)
    
    def getBallPosition(self):
        for entry in self.tiles:
            if self.tiles[entry] == 4:
                return entry
            
    def getPaddlePosition(self):
        for entry in self.tiles:
            if self.tiles[entry] == 3:
                return entry
    
    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
            
            #if len(self.outputList) >= 2 and self.outputList[-2] == -1 and self.outputList[-1] == 0:
            #    print(idx, decoded)

            if decoded[0] == 99:
                print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                #if arg1 + arg2 in [98,59,85]: print(str(arg1) + " + " + str(arg2) + " = " + str(arg1+arg2))
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                #if arg1 * arg2 in [98,59,85]: print(str(arg1) + " * " + str(arg2) + " = " + str(arg1*arg2))
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # break here and wait for updated input from extern
                yield 1
                var = self.input
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                #print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [7]:
#task 1
opcodeMgr=OpCodeManager(opcodeInput.copy())
game = ArcadeGame(opcodeMgr)
game.run(0)

Finished!


In [ ]:
game.processOutput()

In [12]:
numBlockTiles=0
for tile in game.tiles.values():
    if tile == 2:
        numBlockTiles += 1
print(numBlockTiles)

226


In [121]:
#task 2
def convertInput(inputStr):
    if inputStr == 'a':
        return -1
    elif inputStr == 's':
        return 0
    elif inputStr == 'd':
        return 1
    elif inputStr == 'E':
        print("Exiting!")
        assert(False)
    else:
        return convertInput(input("Repeat input: "))

class Arcadebot:
    #def __init__(self):
        
    def nextInput(self,game):
        ballPos = game.getBallPosition()
        paddlePos = game.getPaddlePosition()
        
        if ballPos[0] < paddlePos[0]:
            return -1
        if ballPos[0] > paddlePos[0]:
            return 1
        return 0
    
    
opcodeMgr=OpCodeManager(opcodeInput.copy())
opcodeMgr[0] = 2
game = ArcadeGame(opcodeMgr)

from IPython.display import clear_output

bot = Arcadebot()

for i,step in enumerate(game.run(0)):
    game.processOutput()
    game.visualizeTiles()
    print("Score: "+str(game.score))
    #game.input=convertInput(input("New joystick position: "))
    game.input=bot.nextInput(game)
    clear_output()
    #game.resetTiles()
game.processOutput()
print("Final score: " + str(game.score))

Finished!
Final score: 10800
